In [1]:
%matplotlib ipympl
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import os

from physoce import tseries as ts

from ADCP import rditext_to_dataset

### Load data

#### Rover

In [2]:
# Load data
rover_csv_file = 'data/Rover_II_Current_Mag_Hourly_Avg_pad_2018.csv'
df = pd.read_csv(rover_csv_file,parse_dates=[7])

# Datetime index
df = df.set_index('Date_time_R')

# Create u and v components with units m/s
df['u'] = df['Easting (cm/sec)']/100
df['v'] = df['Northing (cm/sec)']/100

#### ADCP

In [3]:
ds = xr.open_dataset('data/MBARI_StationM_ADCP_201711_201811.nc')

### Resample to hourly values

#### Define date range

In [4]:
t1 = np.datetime64('2017-11-12T00:00:00')
t2 = np.datetime64('2018-10-17T00:00:00')

#### ADCP

In [5]:
dsr = ds.resample({'time':'1H'}).mean()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/xarray/core/nanops.py:161: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [6]:
ti, = np.where((dsr['time'] >= t1) & (dsr['time'] <= t2))
dsf = dsr.isel(time = ti)

#### Rover

In [7]:
dfr = df.resample('1H').mean()
dff = dfr.loc[(dfr.index >= t1) & (dfr.index <= t2)].interpolate(limit=2)

## Plot comparisons

In [12]:
plt.figure(figsize=(7.5,4))
plt.plot(dff.index,dff['u'])
plt.plot(dsf['time'],dsf['Eas'][:,1]+0.1,'-')
yl = plt.ylim()

plt.figure(figsize=(7.5,4))
plt.plot(dff.index,dff['v'])
plt.plot(dsf['time'],dsf['Nor'][:,1]+0.1,'-')
yl = plt.ylim()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [13]:
plt.figure(figsize=(7.5,4))
plt.plot(dff.index,ts.pl64(dff['u']))
plt.plot(dsf['time'],ts.pl64(dsf['Eas'][:,1]),'-')
yl = plt.ylim()
plt.legend(['Rover','ADCP'])

plt.figure(figsize=(7.5,4))
plt.plot(dff.index,ts.pl64(dff['v']))
plt.plot(dsf['time'],ts.pl64(dsf['Nor'][:,1]),'-')
yl = plt.ylim()
plt.legend(['Rover','ADCP'])

FigureCanvasNbAgg()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:251: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  in1zpadded[sc] = in1.copy()


FigureCanvasNbAgg()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:251: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  in1zpadded[sc] = in1.copy()


In [14]:
plt.figure()
plt.plot(dff['u'],dsf['Eas'][:,1],'.')
plt.xlabel('Rover')
plt.ylabel('ADCP')

plt.figure()
plt.plot(dff['v'],dsf['Nor'][:,1],'.')
plt.xlabel('Rover')
plt.ylabel('ADCP')

FigureCanvasNbAgg()

FigureCanvasNbAgg()

Text(0, 0.5, 'ADCP')

In [15]:
resu = stats.linregress(ts.pl64(dff['u'])[::33],ts.pl64(dsf['Eas'][:,1])[::33])
resv = stats.linregress(ts.pl64(dff['v'])[::33],ts.pl64(dsf['Nor'][:,1])[::33])

print(resu)
print(resv)

plt.figure()
plt.plot(ts.pl64(dff['u'])[::33],ts.pl64(dsf['Eas'][:,1])[::33],'.')
plt.xlabel('Rover')
plt.ylabel('ADCP')

plt.figure()
plt.plot(ts.pl64(dff['v'])[::33],ts.pl64(dsf['Nor'][:,1])[::33],'.')
plt.xlabel('Rover')
plt.ylabel('ADCP')

LinregressResult(slope=nan, intercept=nan, rvalue=nan, pvalue=nan, stderr=nan)
LinregressResult(slope=nan, intercept=nan, rvalue=nan, pvalue=nan, stderr=nan)


/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:251: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  in1zpadded[sc] = in1.copy()


FigureCanvasNbAgg()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:251: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  in1zpadded[sc] = in1.copy()


FigureCanvasNbAgg()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:251: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  in1zpadded[sc] = in1.copy()


Text(0, 0.5, 'ADCP')

In [20]:
dsf['Eas'][:,1][::33]

<xarray.DataArray 'Eas' (time: 247)>
array([0.040333, 0.062167, 0.03775 , ..., 0.006167, 0.012167, 0.007833])
Coordinates:
  * time     (time) datetime64[ns] 2017-11-12 ... 2018-10-16T06:00:00